In [1]:
import json, pickle
from collections import Counter
import warnings

import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.base import clone
from scipy.sparse import vstack, hstack
from scipy.stats.stats import spearmanr
from scipy.stats.stats import kendalltau
from tqdm import tqdm

warnings.filterwarnings("ignore")

# Данные

In [2]:
players = pickle.load(open('chgk/players.pkl', 'rb'))
tournaments = pickle.load(open('chgk/tournaments.pkl', 'rb'))

In [3]:
players = pd.DataFrame(players.values()).set_index("id")
players.head()

,name,patronymic,surname
id,,,
1,Алексей,None,Абабилов
10,Игорь,,Абалов
11,Наталья,Юрьевна,Абалымова
12,Артур,Евгеньевич,Абальян
13,Эрик,Евгеньевич,Абальян


In [4]:
tournaments = pd.DataFrame(tournaments.values()).set_index("id")
tournaments.head()

,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty
id,,,,,,,,
1,Чемпионат Южного Кавказа,2003-07-25T00:00:00+04:00,2003-07-27T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/1,[],None,None
2,Летние зори,2003-08-09T00:00:00+04:00,2003-08-09T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/1,[],None,None
3,Турнир в Ижевске,2003-11-22T00:00:00+03:00,2003-11-24T00:00:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None
4,Чемпионат Украины. Переходной этап,2003-10-11T00:00:00+04:00,2003-10-12T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None
5,Бостонское чаепитие,2003-10-10T00:00:00+04:00,2003-10-13T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None


In [9]:
tournaments['dateStart'] = pd.to_datetime(tournaments.dateStart, utc=True)
tournaments = tournaments[tournaments.dateStart.dt.year >= 2019]
print(tournaments.shape)
train_tournaments = tournaments[tournaments.dateStart.dt.year == 2019]
test_tournaments = tournaments[tournaments.dateStart.dt.year == 2020]
print(train_tournaments.shape, test_tournaments.shape)

(1109, 8)
(689, 8) (416, 8)


In [11]:
results = pickle.load(open('chgk/results.pkl', 'rb'))

In [25]:
def get_players_in_tournament(tournament_id):
    l = [[member['player']['id'] for member in team['teamMembers']] for team in results[tournament_id]]
    return [item for sublist in l for item in sublist]